In [28]:
import pandas as pd
import os
location = str(os.getcwd())

In [23]:
f = (location[:-12] + "Articles\\articles_cleaned.csv")
df = pd.read_csv(f)
df = df.to_dict("index")
print(len(df))

11768


In [7]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
stemmer = PorterStemmer()
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wilmd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for word in text:
        word.replace("\n", "")
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [9]:
df_clean = []
print(len(df))
for i in range(len(df)):
    body = df[i]["body"]
    if len(str(body)) > 100:
        if "rape" in body or "sexual" in body or "assault" in body or "misconduct" in body or "harassment" in body:
            df_clean.append(df[i])
print(len(df_clean))

11768
10884


In [10]:
df = pd.DataFrame(df_clean)
processed_docs = df['body'].map(preprocess)
processed_docs[:2]

0    [year, offic, women, affair, work, improv, liv...
1    [third, time, offic, student, associ, administ...
Name: body, dtype: object

In [11]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 accomplish
1 achiev
2 addit
3 address
4 affair
5 aim
6 alex
7 alyc
8 anniversari
9 assault
10 assensoh


In [12]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [13]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [14]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [15]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    from tqdm import tqdm

    for num_topics in tqdm(range(start, limit, step)):
        model = gensim.models.LdaMulticore(corpus = corpus, num_topics = num_topics, id2word = dictionary, passes=2, workers=2)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=bow_corpus, texts=processed_docs, start=2, limit=40, step=1)

  0%|                                                                                                                                             | 0/38 [00:00<?, ?it/s]

2


  3%|███▌                                                                                                                                 | 1/38 [01:19<48:52, 79.27s/it]

3


  5%|███████                                                                                                                              | 2/38 [02:40<48:23, 80.64s/it]

...

 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████       | 36/38 [50:02<03:46, 113.04s/it]

38


 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 37/38 [52:06<01:56, 116.18s/it]

39


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [54:16<00:00, 85.69s/it]


In [16]:
coherence_values

[0.3201158182889358,
 0.3790325209874615,
 0.3710416963143046,
 0.33374569238383345,
 0.3886366105043175,
 0.36453820412799137,
 0.3791220873199293,
 0.3727714668031806,
 0.39832114734486596,
 0.39839163422759183,
 0.3833009113960846,
 0.37289709096124596,
 0.3895807344479638,
 0.37447219440319135,
 0.38517280895149847,
 0.37923962117280074,
 0.3775245456317511,
 0.38245670411852667,
 0.3938980780542207,
 0.3724035519617704,
 0.360581829142865,
 0.38022464229826364,
 0.3717927295253379,
 0.3791448077910565,
 0.38967152753394063,
 0.3840436454085743,
 0.3565104272238512,
 0.3688240014413574,
 0.37116555702081894,
 0.37012673063418744,
 0.37524578020969535,
 0.38077108822202443,
 0.3727897799178545,
 0.36828127253793586,
 0.3943330241410956,
 0.3705832790683772,
 0.3973410374835486,
 0.38100953705536345]

In [18]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=12, id2word=dictionary, passes=2, workers=2)

In [20]:
topic_df = []
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    topic_df.append({"index": idx, "topic_info": topic})
    
pd.DataFrame(topic_df).to_csv('topics_info.csv')

Topic: 0 
Words: 0.007*"state" + 0.006*"right" + 0.005*"unit" + 0.004*"american" + 0.004*"group" + 0.004*"work" + 0.004*"countri" + 0.004*"nation" + 0.003*"citi" + 0.003*"campu"
Topic: 1 
Words: 0.010*"state" + 0.006*"madison" + 0.006*"campu" + 0.005*"wisconsin" + 0.004*"senat" + 0.004*"health" + 0.004*"report" + 0.004*"team" + 0.003*"vote" + 0.003*"includ"
Topic: 2 
Words: 0.016*"athlet" + 0.008*"coach" + 0.008*"team" + 0.007*"minnesota" + 0.006*"player" + 0.006*"report" + 0.005*"footbal" + 0.004*"school" + 0.004*"state" + 0.004*"presid"
Topic: 3 
Words: 0.005*"work" + 0.005*"hous" + 0.005*"colleg" + 0.005*"think" + 0.004*"go" + 0.004*"member" + 0.004*"school" + 0.004*"commun" + 0.004*"campu" + 0.004*"know"
Topic: 4 
Words: 0.007*"work" + 0.007*"campu" + 0.006*"faculti" + 0.006*"polici" + 0.006*"school" + 0.005*"report" + 0.005*"research" + 0.005*"depart" + 0.005*"offic" + 0.005*"commun"
Topic: 5 
Words: 0.017*"women" + 0.006*"commun" + 0.006*"violenc" + 0.005*"cultur" + 0.005*"think"

In [21]:
for index, score in sorted(lda_model[bow_corpus[7]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6442173719406128	 
Topic: 0.005*"play" + 0.005*"film" + 0.005*"know" + 0.004*"go" + 0.004*"think" + 0.004*"music" + 0.003*"work" + 0.003*"game" + 0.003*"want" + 0.003*"love"

Score: 0.170234814286232	 
Topic: 0.008*"women" + 0.008*"think" + 0.005*"want" + 0.004*"know" + 0.004*"work" + 0.004*"thing" + 0.003*"go" + 0.003*"issu" + 0.003*"world" + 0.003*"right"

Score: 0.13471761345863342	 
Topic: 0.005*"work" + 0.005*"hous" + 0.005*"colleg" + 0.005*"think" + 0.004*"go" + 0.004*"member" + 0.004*"school" + 0.004*"commun" + 0.004*"campu" + 0.004*"know"

Score: 0.018177516758441925	 
Topic: 0.017*"women" + 0.006*"commun" + 0.006*"violenc" + 0.005*"cultur" + 0.005*"think" + 0.005*"abus" + 0.004*"campu" + 0.004*"right" + 0.004*"know" + 0.004*"experi"

Score: 0.01671760529279709	 
Topic: 0.021*"report" + 0.016*"polic" + 0.012*"campu" + 0.010*"crime" + 0.007*"accord" + 0.007*"incid" + 0.006*"victim" + 0.006*"commun" + 0.006*"offic" + 0.005*"depart"

Score: 0.01534430030733347	 
Topic: 0

Saving results to dataframe


In [24]:
df_clean = []

from tqdm import tqdm

for i in tqdm(range(len(df))):
    item = df[i]
    body = item["body"]
    url = item["url"]
    title = item["title"]
    location = item["location"]
    date = item["date"]
    processed_body = preprocess(str(body))
    bow_index = dictionary.doc2bow(processed_body)
    
    topic_list = []
    j = 0
    for index, score in sorted(lda_model[bow_index], key=lambda tup: -1*tup[1]):
        topic_list.append({"Score": score, "Topic": index, "Topic_terms": lda_model.print_topic(index, 10)})
        if j == 0:
            lead_topic = index
        j += 1
    
    if len(str(body)) > 100:
        if " rape " in body or " sexual " in body or " assault " in body or " misconduct " in body or " harassment " in body or " harass " in body:
            df_clean.append({"url": url, "body": body, "title": title, "location": location, "date": date, "topic scores": topic_list, "top_topic": lead_topic})
print(len(df_clean))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11768/11768 [02:40<00:00, 73.38it/s]

8150


In [25]:
pd.DataFrame(df_clean).to_csv('articles_with_topic.csv')

Deriving meaning from topics

In [30]:
f = (location + "\\articles_with_topic.csv")
df_clean = pd.read_csv(f)
df_clean = df_clean.to_dict("index")
print(len(df_clean))

8150


In [32]:
import ast

sorted_articles = []
for i in range(0,12):
    sorted_articles.append([])

for j in range(len(df_clean)):
    item = df_clean[j]
    topic_scores = ast.literal_eval(item["topic scores"])
    topic_id = topic_scores[0]["Topic"]
    sorted_articles[topic_id].append(item["url"])

In [35]:
import random
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print(len(sorted_articles[idx]))
    for item in random.sample(sorted_articles[idx], 20):
        print(item)
    print("==============================")

Topic: 0 
Words: 0.007*"state" + 0.006*"right" + 0.005*"unit" + 0.004*"american" + 0.004*"group" + 0.004*"work" + 0.004*"countri" + 0.004*"nation" + 0.003*"citi" + 0.003*"campu"
258
https://mndaily.com/199316/opinion/gfaktfgvwuc2ijy/
https://mndaily.com/237407/uncategorized/disaster-revisited-darfur/
https://dbknews.com/2017/09/19/harvey-irma-foreign-crises-racism/
https://mndaily.com/186785/uncategorized/net-our-mailbox-ke/
https://www.idsnews.com/article/2013/01/tales-of-the-tsa
https://dbknews.com/2016/08/12/article_aade80ae-9312-58f7-8fc5-27d066f363b5-html/
https://dbknews.com/2016/08/12/article_323d55e2-87e5-5179-9337-0f0b028696df-html/
https://stanforddaily.com/2011/12/07/op-ed-wake-up-the-case-for-abolishing-the-death-penalty/
https://stanforddaily.com/2018/03/15/prison-labor-modern-slavery/
https://dbknews.com/0999/12/31/arc-aq6rhx66krbdfccwupxjjv6ozy/
https://dbknews.com/2006/04/25/article_2ab3b1b9-de44-5162-b959-68dba5b2239b-html/
https://dbknews.com/2017/05/02/north-korea-po

In [5]:
import random
for i in range(0,12):
    print(i)
    print(len(sorted_articles[i]))
    for link in random.sample(sorted_articles[i], 20):
        print(link)

0
1250
https://mndaily.com/209121/arts-entertainment/comedy/book-review-hilarity-ensues-tucker-max/
https://dbknews.com/2016/09/21/are-songs-the-next-target-in-the-fight-for-political-correctness/
https://dbknews.com/2020/12/02/diego-maradona-human-rights-sexual-assault/
https://mndaily.com/238813/uncategorized/playboy-representatives-arent-fazed-protests/
https://www.idsnews.com/article/2012/03/reform-not-discipline-and-punish
https://www.idsnews.com/article/2015/12/editorial-porn-star-james-deen-accused-of-raping-stoya-and-others
https://stanforddaily.com/2018/10/05/me-asl-a-moral-member-for-the-highest-court/
https://www.dailycardinal.com/article/2018/01/stars-use-golden-globes-platform-to-further-equality-movement
https://dbknews.com/0999/12/31/arc-7cthjydgtncmjdy74skpqwpsda/
https://www.idsnews.com/article/2018/04/column-getz-bush-042418
https://www.idsnews.com/article/2016/02/kanye-west-tweeter-extrodinaire
https://mndaily.com/232831/uncategorized/sergeant-convicted-raping-six-tr